In [1]:
import numpy as np

In [17]:
def find_max_rank( ranks ):
    """ Find the highest ranked item """
    max_rank  = 0
    for i in len(ranks):
        if ranks[i] > max_rank:
            max_rank = ranks[i]
    return max_rank

In [18]:
def find_max_rating_score(user_item_ratings):
    max_item_score = 0 
    
    for i in range(len(user_item_ratings)):
        for j in range(len(user_item_ratings[i])):
                if user_item_ratings[i][j] > max_item_score:
                    max_item_score = user_item_ratings[i][j]
    return max_item_score

$$ ||u|| = \sqrt{u_{0}^{2}+u_{1}^{2}......u_{n}^{2}} $$

In [22]:
def norm_vector(vector):
    output = 0
    for i in len(vector):
        output = output + ( vector[i] ** 2 ) 

$$ {U_{i}^{T}\cdot V_{j}} $$

In [ ]:
def user_feature_dot_item_feature(user_feature_vector, item_feature_vector):
    return np.dot(np.transpose(user_feature_vector), item_feature_vector)

$$ || U_{i} || \cdot || V_{j} || $$

In [ ]:
def norm_user_feature_dot_norm_item_feature(user_feature_vector, item_feature_vector):
    return np.dot(norm_vector(user_feature_vector), norm_vector(item_feature_vector))

#### Estimate Rank Function

$$ x_{j} = \sum \limits _{i=1} ^{m} \alpha_{i}U_{i}^{T}\cdot V_{j} $$

In [15]:
def estimate_rank( alpha_coefficents, user_feature_vectors, item_feature_vector ):
    """Calculate the estimated rank of the item based on the user_feature_vectors and the item_feature_vector"""
    rank = 0

    for i in range( 0, len(user_feature_vectors)):
        rank = rank + np.multiply(alpha_coefficents[i], np.dot(user_feature_vectors[i], item_feature_vector))
    return rank

#### Zipf Exponent

$$ s = 1 + n \left( \begin{array}{cc} \sum \limits _{i=1} ^{n} ln \frac{x_{i}}{x_{max}}  \end{array}\right)^{-1}  $$

In [ ]:
def zipf_exponent(alpha_coefficents, user_feature_vectors, item_feature_vectors):
    ranks = []
    for i in range( item_feature_vectors ):
        ranks.append(estimate_rank(alpha_coefficents, user_feature_vectors, item_feature_vectors[i]))
    max_rank = find_max_rank(ranks)

    sum_ranks = 0
    for j in len(ranks):
        sum_ranks = sum_ranks + np.log(np.divide(ranks[j], max_rank))

    return 1 + np.multiply(len(item_feature_vectors), np.power(sum_ranks, -1))

### Loss Function

$$ L =\sum \limits _{i=1} ^{m}\sum \limits _{j=1} ^{n} \left( \begin{array}{cc} \frac{R_{ij}}{R_{max}} - \frac{U_{i}^{T}\cdot V_{j}}{ || U_{i} || \cdot || V_{j} || } \end{array}\right)^{2} - \beta \left( \begin{array}{cc} 1 + n \left( \begin{array}{cc} \sum \limits _{i=1} ^{n} ln \frac{x_{i}}{x_{max}}  \end{array}\right)^{-1} \end{array}\right) $$

In [23]:
def loss_function(user_items_ratings_scores, user_feature_vectors, item_feature_vectors, alpha_coefficents,
                  beta_coefficent):
    max_item_score = find_max_rating_score(user_items_ratings_scores)

    matrix_loss = 0
    for i in range(m):
        for i in range(n):
            matrix_loss = matrix_loss + np.divide(user_items_ratings_scores[i][j], max_item_score) - \
                          np.divide(user_feature_dot_item_feature(user_feature_vectors[i], item_feature_vector[i]),
                                    norm_user_feature_dot_norm_item_feature(user_feature_vectors[i],
                                                                            item_feature_vector[i]))
            
    matrix_loss = np.power(matrix_loss, 2)
    
    #Penalize by ZipF exponent
    matrix_loss = matrix_loss - np.multiply(beta_coefficent, zipf_exponent(alpha_coefficents, user_feature_vectors,
                                                                           item_feature_vectors))
    
    return matrix_loss


$$ U_{i} = U_{i} - \mathcal{E} \cdot \left( \begin{array}{cc}   \frac{n}{log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2}} \end{array}\right)$$

$$ \frac{n}{log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2}} $$

$$ \frac{n}{\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)} $$